# CNN #3

In [4]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

# fix dimension ordering issue
from keras import backend as K
K.set_image_dim_ordering('th')

from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

seed = 28
np.random.seed(seed)

In [5]:
floydhub_dir = "/floyd/input/volcanoes_venus"

# Load data - Floydhub
data_train = pd.read_csv("/floyd/input/volcanoes_venus/train_images.csv", header=None)
labels_train = pd.read_csv("/floyd/input/volcanoes_venus/train_labels.csv")
data_test = pd.read_csv("/floyd/input/volcanoes_venus/test_images.csv", header=None)
labels_test = pd.read_csv("/floyd/input/volcanoes_venus/test_labels.csv")

# Load data - Local
#data_train = pd.read_csv("data/train_images.csv", header=None)
#labels_train = pd.read_csv("data/train_labels.csv")
#data_test = pd.read_csv("data/test_images.csv", header=None)
#labels_test = pd.read_csv("data/test_labels.csv")

print("Train-data rows: {}, Train-data columns: {}".format(data_train.shape[0], data_train.shape[1]))
print("Train-labels rows: {}, Train-labels columns: {}".format(labels_train.shape[0], labels_train.shape[1]))
print("Test-data rows: {}, Test-data columns: {}".format(data_test.shape[0], data_test.shape[1]))
print("Test-labels rows: {}, Test-labels columns: {}".format(labels_test.shape[0], labels_test.shape[1]))

Train-data rows: 7000, Train-data columns: 12100
Train-labels rows: 7000, Train-labels columns: 4
Test-data rows: 2734, Test-data columns: 12100
Test-labels rows: 2734, Test-labels columns: 4


In [6]:
# Prep data for modeling
X_train = np.array(data_train.values).astype("float32")
y_train = np.array(labels_train["Volcano?"].values).astype("float32")
X_test = np.array(data_test.values).astype("float32")
y_test = np.array(labels_test["Volcano?"].values).astype("float32")

# Shape to include channel dim for Conv2D
X_train = X_train.reshape(X_train.shape[0], 1, 110, 110)
X_test = X_test.reshape(X_test.shape[0], 1, 110, 110)

# Normalize input
X_train = X_train / 255
X_test = X_test / 255

# Up-sample to balance target class
X_upsampled, y_upsampled = resample(X_train[y_train == 1], y_train[y_train == 1], replace=True,
                                   n_samples=X_train[y_train == 0].shape[0], random_state=seed)

X_train_up = np.vstack((X_train[y_train == 0], X_upsampled))
y_train_up = np.hstack((y_train[y_train == 0], y_upsampled))

y_pred = np.zeros(y_train_up.shape[0])
print("New class distribution:", np.mean(y_pred == y_train_up))

print("X_train_up shape:", X_train_up.shape)
print("y_train_up shape:", y_train_up.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

New class distribution: 0.5
X_train_up shape: (12000, 1, 110, 110)
y_train_up shape: (12000,)
X_test shape: (2734, 1, 110, 110)
y_test shape: (2734,)


In [7]:
input_dim = (X_train_up.shape[1], X_train_up.shape[2], X_train_up.shape[3])
epochs = 25
print("Input dim:", input_dim)

Input dim: (1, 110, 110)


In [8]:
def create_model(input_dim, output_dim, epochs=25, lrate=0.01):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=input_dim, padding="same", activation="relu",
                    kernel_constraint=maxnorm(3)))
    model.add(Dropout(rate=0.2))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same",
                     kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu", kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, activation="sigmoid"))

    decay = lrate / epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    
    model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
    return model

In [9]:
model = create_model(input_dim, 1, epochs=epochs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 110, 110)      320       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 110, 110)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 110, 110)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 55, 55)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 96800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               49562112  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [10]:
model.fit(X_train_up, y_train_up, validation_data=(X_test, y_test), epochs=epochs, batch_size=200, verbose=1)

Train on 12000 samples, validate on 2734 samples
Epoch 1/25
12000/12000 [==============================] - 22s 2ms/step - loss: 0.6902 - acc: 0.5412 - val_loss: 0.6285 - val_acc: 0.8409
Epoch 2/25
12000/12000 [==============================] - 19s 2ms/step - loss: 0.6303 - acc: 0.6776 - val_loss: 0.4845 - val_acc: 0.8745
Epoch 3/25
12000/12000 [==============================] - 19s 2ms/step - loss: 0.4455 - acc: 0.8137 - val_loss: 0.5421 - val_acc: 0.7992
Epoch 4/25
12000/12000 [==============================] - 19s 2ms/step - loss: 0.3576 - acc: 0.8571 - val_loss: 0.4444 - val_acc: 0.8204
Epoch 5/25
12000/12000 [==============================] - 19s 2ms/step - loss: 0.3288 - acc: 0.8633 - val_loss: 0.2992 - val_acc: 0.9075
Epoch 6/25
12000/12000 [==============================] - 19s 2ms/step - loss: 0.2922 - acc: 0.8823 - val_loss: 0.2684 - val_acc: 0.9217
Epoch 7/25
12000/12000 [==============================] - 19s 2ms/step - loss: 0.2657 - acc: 0.8947 - val_loss: 0.2828 - val_acc:

In [11]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline accuracy: {:.2f}%".format(100 * scores[1]))
print("Baseline error rate: {:.2f}%".format(100 * (1 - scores[1])))

Baseline accuracy: 95.39%
Baseline error rate: 4.61%


In [12]:
preds = model.predict_classes(X_test)

In [13]:
print("Test accuracy:", accuracy_score(y_test, preds))
print("\nConfusion matrix:\n")
print(confusion_matrix(y_test, preds))
print("\nClassification report:\n")
print(classification_report(y_test, preds))

Test accuracy: 0.9539136795903438

Confusion matrix:

[[2247   53]
 [  73  361]]

Classification report:

             precision    recall  f1-score   support

        0.0       0.97      0.98      0.97      2300
        1.0       0.87      0.83      0.85       434

avg / total       0.95      0.95      0.95      2734

